In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from encoder import Encoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

In [7]:
ls src

basis_expansions.py    dftransformers.py
categorical_plot.py    regression_helpers.py


In [8]:
data = pd.read_csv('Fraud_Data.csv')

In [9]:
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [10]:
ip_country = pd.read_csv('IpAddress_to_Country.csv')

In [11]:
ip_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
lower_bound_ip_address    138846 non-null float64
upper_bound_ip_address    138846 non-null int64
country                   138846 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [12]:
diff = (ip_country['upper_bound_ip_address'] - data['ip_address'][0])
diff[diff > 0].idxmin()

4554

In [13]:
print(ip_country['upper_bound_ip_address'][4554] > data['ip_address'][0])
print(ip_country['lower_bound_ip_address'][4554] < data['ip_address'][0])

True
True


In [29]:
def find_ip_country(ip_address):
    diff = ip_country['upper_bound_ip_address'] - ip_address
    if len(diff[diff > 0]) == 0:
        return '[not specified]'
    country_idx = diff[diff > 0].idxmin()
    return ip_country.loc[country_idx, 'country']

In [15]:
ip_country.loc[4554, 'country']

'Japan'

In [16]:
len(ip_country.loc[ip_country['upper_bound_ip_address'] > 3758096383, 'upper_bound_ip_address'])

0

In [17]:
data.loc[data['ip_address'] == 3840542443.91396]

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0


In [18]:
max(ip_country['upper_bound_ip_address'])

3758096383

In [19]:
data['country'] = data['ip_address'].apply(find_ip_country)

In [39]:
data['signup_dt'] = pd.to_datetime(data['signup_time'])
data['purchase_dt'] = pd.to_datetime(data['purchase_time'])

In [40]:
data.loc[0, 'purchase_dt'] - data.loc[0, 'signup_dt']

Timedelta('52 days 03:51:22')

In [47]:
def dt_diff(row):
    return row['purchase_dt'] - row['signup_dt']

data['dt_diff'] = data.apply(dt_diff, axis=1)

52

In [82]:
data['payment_lag_days'] = data['dt_diff'].apply(lambda x: x.days)

In [83]:
data.columns

Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'source', 'browser', 'sex', 'age', 'ip_address', 'class',
       'country', 'signup_dt', 'purchase_dt', 'dt_diff', 'payment_lag_days'],
      dtype='object')

In [123]:
df.columns

Index(['purchase_value', 'source', 'browser', 'sex', 'age', 'country',
       'payment_lag_days'],
      dtype='object')

In [142]:
cols_to_standardize = ['purchase_value', 'age', 'payment_lag_days']
std_columns = [col + '_std' for col in cols_to_standardize]
ss = StandardScaler()
std_df = pd.DataFrame(ss.fit_transform(df[cols_to_standardize].astype(float)),
                             columns=std_columns)

In [143]:
data = data.merge(std_df, left_index=True, right_index=True)

In [144]:
cols_to_drop = (['user_id', 'signup_time', 'purchase_time',
                 'device_id', 'ip_address', 'signup_dt', 
                 'purchase_dt', 'class', 'dt_diff',
                 'purchase_value', 'age', 'payment_lag_days'])
df = data.drop(cols_to_drop, axis=1)

In [167]:
enc = Encoder(columns=['source', 'browser', 'sex', 'country'], thresh=5)

In [168]:
for x in X.columns:
    print(x)

purchase_value_std
age_std
payment_lag_days_std
source_SEO
source_Ads
source_Direct
browser_Chrome
browser_IE
browser_Safari
browser_FireFox
browser_Opera
sex_M
sex_F
country_United States
country_[not_specified]
country_China
country_Japan
country_United Kingdom
country_Korea Republic of
country_Germany
country_France
country_Canada
country_Brazil
country_Australia
country_Italy
country_Russian Federation
country_Netherlands
country_India
country_Taiwan; Republic of China (ROC)
country_Romania
country_Mexico
country_Sweden
country_Spain
country_South Africa
country_Switzerland
country_Poland
country_Argentina
country_Indonesia
country_Norway
country_Colombia
country_Turkey
country_Viet Nam
country_Denmark
country_Hong Kong
country_Finland
country_Austria
country_Ukraine
country_Chile
country_Belgium
country_Iran (ISLAMIC Republic Of)
country_Egypt
country_Czech Republic
country_Thailand
country_New Zealand
country_Israel
country_Saudi Arabia
country_Venezuela
country_Ireland
country_E

In [169]:
X = enc.fit_transform(df)
y = data['class']

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)

In [175]:
model = LogisticRegression(penalty='l2', C=2, 
                           solver='lbfgs', max_iter=500, verbose=100, n_jobs=None)

In [176]:
model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.9s finished


LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=100, warm_start=False)

In [178]:
model.coef_

array([[ 0.01310002,  0.01853634, -1.03838375, -0.08774292, -0.06358927,
         0.10032194,  0.05023128, -0.07732623, -0.02271322,  0.02650873,
        -0.02771081, -0.00926455, -0.0417457 ,  0.31459149,  0.14371071,
         0.22834349,  0.3350828 ,  0.45727717,  0.23958315,  0.00933153,
         0.35373264,  0.51823848,  0.27336275,  0.33845791,  0.12056556,
         0.18222602,  0.12794527,  0.55560088,  0.10420751,  0.08197283,
         0.6442507 ,  0.60065827, -0.02890714,  0.17181522,  0.27193206,
        -0.34194049,  0.34526131,  0.23059941,  0.72566231, -0.03599786,
         0.02978633,  0.09114816,  0.83414449,  0.70619549,  0.15210526,
        -0.00485314,  0.44715671,  0.71293074,  0.72915222,  0.48273199,
         0.69806132,  0.19139115, -0.16303503,  1.09639077, -0.68149057,
         0.96065778,  0.69887941,  1.07062811, -0.34439325,  0.68678805,
        -0.57658274,  0.20847509, -0.23309458, -0.29505896, -0.5619807 ,
        -0.00633072, -1.6759975 , -0.31623258,  0.3

In [180]:
y_pred = model.predict_proba(X_test)

In [188]:
y_pred

array([[0.94666867, 0.05333133],
       [0.95804462, 0.04195538],
       [0.14131481, 0.85868519],
       ...,
       [0.13903039, 0.86096961],
       [0.95561896, 0.04438104],
       [0.95606056, 0.04393944]])

In [182]:
log_loss(y_test, y_pred)

0.271886119791659

In [183]:
from sklearn.ensemble import GradientBoostingClassifier

In [184]:
gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)

In [185]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [187]:
y_pred = gb.predict_proba(X_test)